In [270]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [271]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

In [272]:
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
submission=pd.read_csv('../input/titanic/gender_submission.csv')

In [273]:
train.head()

In [274]:
train.isnull().sum()

In [275]:
train['Pclass'].value_counts()

In [276]:
train['Cabin'].unique()

In [277]:
train = train.drop(columns = 'Cabin')
test = test.drop(columns = 'Cabin')
train = train.drop(columns = 'PassengerId')
test = test.drop(columns = 'PassengerId')

In [278]:
train['Embarked'].value_counts()

In [279]:
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Embarked'] = train['Embarked'].fillna('S')
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Embarked'] = test['Embarked'].fillna('S')

In [280]:
sns.barplot(data=train,x='Pclass',y='Survived')

In [281]:
train['Pclass_1'] = (train['Pclass']==1)
train['Pclass_2'] = (train['Pclass']==2)
train['Pclass_3'] = (train['Pclass']==3)

test['Pclass_1'] = (test['Pclass']==1)
test['Pclass_2'] = (test['Pclass']==2)
test['Pclass_3'] = (test['Pclass']==3)

In [282]:
train = train.drop(columns = 'Pclass')
test = test.drop(columns = 'Pclass')

In [283]:
train['Title'] = train['Name'].str.extract("([A-Za-z]+)\.", expand=False)
train['Title'].value_counts()

In [284]:
test['Title'] = test['Name'].str.extract("([A-Za-z]+)\.", expand=False)

In [285]:
sns.barplot(data=train,x='Title',y='Survived')

In [286]:
train['Title'] = train['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
train['Title'] = train['Title'].replace(['Jonkheer', 'Master'], 'Master')
train['Title'] = train['Title'].replace(['Don', 'Sir', 'Countess', 'Lady', 'Dona'], 'Royalty')
train['Title'] = train['Title'].replace(['Mme', 'Ms', 'Mrs'], 'Mrs')
train['Title'] = train['Title'].replace(['Mlle', 'Miss'], 'Miss')
train['Title'] = train['Title'].replace(['Mr'], 'Mr')

test['Title'] = test['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
test['Title'] = test['Title'].replace(['Jonkheer', 'Master'], 'Master')
test['Title'] = test['Title'].replace(['Don', 'Sir', 'Countess', 'Lady', 'Dona'], 'Royalty')
test['Title'] = test['Title'].replace(['Mme', 'Ms', 'Mrs'], 'Mrs')
test['Title'] = test['Title'].replace(['Mlle', 'Miss'], 'Miss')
test['Title'] = test['Title'].replace(['Mr'], 'Mr')

In [287]:
train['Title_1'] = (train['Title']=='Officer')
train['Title_2'] = (train['Title']=='Master')
train['Title_3'] = (train['Title']=='Royalty')
train['Title_4'] = (train['Title']=='Mrs')
train['Title_5'] = (train['Title']=='Miss')
train['Title_6'] = (train['Title']=='Mr')

test['Title_1'] = (test['Title']=='Officer')
test['Title_2'] = (test['Title']=='Master')
test['Title_3'] = (test['Title']=='Royalty')
test['Title_4'] = (test['Title']=='Mrs')
test['Title_5'] = (test['Title']=='Miss')
test['Title_6'] = (test['Title']=='Mr')

In [288]:
train = train.drop(columns = 'Title')
test = test.drop(columns = 'Title')

In [289]:
train = train.drop(columns = 'Name')
test = test.drop(columns = 'Name')

In [290]:
train.dtypes

In [291]:
train['Sex'].value_counts()

In [292]:
train['male'] = (train['Sex']=='male')
test['male'] = (test['Sex']=='male')

In [293]:
train = train.drop(columns = 'Sex')
test = test.drop(columns = 'Sex')

In [294]:
sns.barplot(data=train,x='Age',y='Survived')

In [295]:
bin = [0, 18, 25, 35, 60, 100]
group_names = ['Baby', 'Youth', 'YoungAdult', 'MiddleAged', 'Senior']
train['AgeGroup'] = pd.cut(train['Age'], bins=bin, labels=group_names)
train['AgeGroup'].value_counts()

In [296]:
sns.barplot(data=train,x='AgeGroup',y='Survived')

In [297]:
test['AgeGroup'] = pd.cut(test['Age'], bins=bin, labels=group_names)
test['AgeGroup'].value_counts()

In [298]:
train = train.drop(columns = 'Age')
test = test.drop(columns = 'Age')

In [299]:
# train['AgeGroup_1'] = (train['AgeGroup'] == 'Baby')
# train['AgeGroup_2'] = (train['AgeGroup'] == 'Senior')

# test['AgeGroup_1'] = (test['AgeGroup'] == 'Baby')
# test['AgeGroup_2'] = (test['AgeGroup'] == 'Senior')

train = train.drop(columns = 'AgeGroup')
test = test.drop(columns = 'AgeGroup')

In [300]:
train['Embarked_1'] = (train['Embarked'] == 'S')
train['Embarked_2'] = (train['Embarked'] == 'C')
train['Embarked_3'] = (train['Embarked'] == 'Q')

test['Embarked_1'] = (test['Embarked'] == 'S')
test['Embarked_2'] = (test['Embarked'] == 'C')
test['Embarked_3'] = (test['Embarked'] == 'Q')

train = train.drop(columns = 'Embarked')
test = test.drop(columns = 'Embarked')

In [301]:
sns.barplot(data=train,x='Fare',y='Survived')

In [302]:
train = train.drop(columns = 'Fare')
test = test.drop(columns = 'Fare')

In [303]:
train['Parch'].value_counts()

In [304]:
sns.barplot(data=train,x='Parch',y='Survived')

In [305]:
train = train.drop(columns = 'Parch')
test = test.drop(columns = 'Parch')

In [306]:
train['SibSp'].value_counts()

In [307]:
sns.barplot(data=train,x='SibSp',y='Survived')

In [308]:
# train['SibSp_0'] = (train['SibSp']==0)
# train['SibSp_1'] = (train['SibSp']==1)
# train['SibSp_2'] = (train['SibSp']==2)
# train['SibSp_3'] = (train['SibSp']==3)
# train['SibSp_4'] = (train['SibSp']==4)
# train['SibSp_5'] = (train['SibSp']==5)
# train['SibSp_8'] = (train['SibSp']==8)

# test['SibSp_0'] = (test['SibSp']==0)
# test['SibSp_1'] = (test['SibSp']==1)
# test['SibSp_2'] = (test['SibSp']==2)
# test['SibSp_3'] = (test['SibSp']==3)
# test['SibSp_4'] = (test['SibSp']==4)
# test['SibSp_5'] = (test['SibSp']==5)
# test['SibSp_8'] = (test['SibSp']==8)

train = train.drop(columns = 'SibSp')
test = test.drop(columns = 'SibSp')

In [309]:
train['Ticket'].unique()

In [310]:
train = train.drop(columns = 'Ticket')
test = test.drop(columns = 'Ticket')

In [311]:
train.head()

In [312]:
Y_train=train['Survived']
X_train=train[list(test)]
X_test=test[list(test)]

print(X_train.shape, Y_train.shape, X_test.shape)

In [313]:
test.isnull().sum()

In [314]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=10, random_state=1)
model.fit(X_train, Y_train)
submission['Survived'] = model.predict(X_test)
submission.to_csv('./titanic_submission.csv', index = False)
submission